In [1]:
import glob
import sys
import subprocess
import numpy as np
import pandas as pd
import os

sys.path.append("/u/lukas.rilling/dev/")

from returnn_training_progress import get_epoch_data
from returnn_training_plot_nb import plot_df
%load_ext autoreload
%autoreload 2

In [2]:
base_dir = "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/"
accuracy_files = str(
    subprocess.check_output(
        f"find {base_dir} -type l -name 'forward_invertibility' -not -path '*/lm*'",
        shell=True,
    ),
    "utf-8",
).split("\n")[:-1]

In [3]:
experiment_group = []
paths = []
df = None
for i,f in enumerate(accuracy_files):
    filename = f + "/output/output.hdf"
    if os.path.exists(filename):
        if df is None:
            df = pd.read_csv(filename)
        else:
            df2 = pd.read_csv(filename)
            df = pd.concat([df, df2])
    
        folders = ["librispeech_glow_asr", "joint_training/default", "joint_training/conformer_coupling", "joint_training/given_alignments"]
        found = False
        for folder in folders:
            if folder in f:
                experiment_group.append(folder)
                found = True
                break
        paths.append(f.replace("forward_invertibility/invertibility", "").replace("/u/lukas.rilling/experiments/glow_tts_asr_v2/output/experiments/librispeech", ""))
        assert len(paths) == len(experiment_group), f"Can't find this experiment group: {f}"

assert len(paths) == len(experiment_group), f"Well wer has length: {len(accuracies)} and experiment_group has length {len(experiment_group)}"
df["Group"] = experiment_group
df["Path"] = paths
df.head(5)

# accuracy_files

,total,mean,var,max,min,Group,Path
0,244390560,0.618195,2.617023e+05,2.852016e+06,0.0,librispeech_glow_asr,/u/lukas.rilling/experiments/glow_tts_asr_v2/a...
0,244390560,0.000001,2.096195e-12,2.357960e-04,0.0,librispeech_glow_asr,/u/lukas.rilling/experiments/glow_tts_asr_v2/a...
0,244390560,0.039957,5.573043e-02,1.344167e+03,0.0,librispeech_glow_asr,/u/lukas.rilling/experiments/glow_tts_asr_v2/a...
0,244390560,0.000023,2.390446e-09,3.259611e-02,0.0,librispeech_glow_asr,/u/lukas.rilling/experiments/glow_tts_asr_v2/a...
0,244390560,0.042055,1.417796e-02,5.968618e+02,0.0,joint_training/default,/u/lukas.rilling/experiments/glow_tts_asr_v2/a...


In [4]:
df["Name"] = df["Path"].str.removeprefix("/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech")
df.rename(columns=lambda x: x.strip(), inplace=True)

In [5]:
print(df[["Group", "Name", "mean", "var", "min", "max"]].to_markdown())

|    | Group                           | Name                                                                                                                       |        mean |              var |   min |            max |
|---:|:--------------------------------|:---------------------------------------------------------------------------------------------------------------------------|------------:|-----------------:|------:|---------------:|
|  0 | librispeech_glow_asr            | /librispeech_glow_asr/pytorch/weak_conformer/asr_dataset/spec_augment_before/glow_not_pretrained/forward_invertibility     | 0.618195    | 261702           |     0 |    2.85202e+06 |
|  0 | librispeech_glow_asr            | /librispeech_glow_asr/pytorch/conformer/asr_dataset/spec_augment_before/glow_enc192_not_pretrained/forward_invertibility   | 1.04137e-06 |      2.09619e-12 |     0 |    0.000235796 |
|  0 | librispeech_glow_asr            | /librispeech_glow_asr/pytorch/conformer/asr_dataset/spec_augmen

In [6]:
index = pd.MultiIndex.from_arrays([experiment_group, accuracy_files], names=("Group", "Experiment"))

df = pd.DataFrame({"Accuracy": accuracies}, index=index)
df["Accuracy [%]"] = df["Accuracy"] * 100
df["Accuracy [%]"] = df["Accuracy [%]"].round(decimals=2)
df = df.drop("Accuracy", axis=1)
df.head(5)

NameError: name 'accuracies' is not defined

In [ ]:
ce_scores = []
ce_dev_scores = []
overfitting = []
mle_scores = []
dp_scores = []
lr = []
finished = []
joint = []
for index, series in df.iterrows():
    data = get_epoch_data((f"{index[1]}training/work/learning_rates").replace("output", "alias"), None)
    breakpoint()
    if data is None or "ce" not in data[list(data.keys())[-1]]["error"]:
        if data is None:
            finished.append(True)
        else:
            finished.append(False)
        ce_scores.append(np.nan)
        ce_dev_scores.append(np.nan)
        mle_scores.append(np.nan)
        dp_scores.append(np.nan)
        overfitting.append(np.nan)
    else:
        last_epoch_data = data[list(data.keys())[-1]]
        finished.append(True)
        ce_scores.append(last_epoch_data["error"]["ce"])
        ce_dev_scores.append(last_epoch_data["error"]["dev_loss_ce"])
        overfitting.append(ce_dev_scores[-1] / ce_scores[-1])

        if "mle" in last_epoch_data["error"]:
            mle_scores.append(last_epoch_data["error"]["mle"])
            dp_scores.append(last_epoch_data["error"]["dp"])
        else:
            mle_scores.append("-")
            dp_scores.append("-")

df["CE"] = ce_scores
df["dev CE"] = ce_dev_scores
df["MLE"] = mle_scores
df["DP"] = dp_scores
df["Joint"] = [True if x != "-" and x != False else False for x in mle_scores]
df["Still running"] = [not x for x in finished]
df["overfitting"] = overfitting
df["Training data available"] = df["CE"] != np.nan

df.head(5)

Accuracy [%]  \
Group                           Experiment                                                         
joint_training/given_alignments /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...         21.30   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...         26.49   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...         27.30   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...         30.09   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...         28.56   

                                                                                          CE  \
Group                           Experiment                                                     
joint_training/given_alignments /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  2.920585   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  2.868420   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  2.664396   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  2.782269   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  2.813527   

                                                                                      dev CE  \
Group                           Experiment                                                     
joint_training/given_alignments /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  2.941674   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  2.858513   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  2.957262   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  2.795637   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  2.825326   

                                                                                         MLE  \
Group                           Experiment                                                     
joint_training/given_alignments /u/lukas.rilling/experiments/glow_tts_asr_v2/ou... -0.797284   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou... -0.802836   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  -0.72108   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou... -0.785492   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou... -0.803716   

                                                                                          DP  \
Group                           Experiment                                                     
joint_training/given_alignments /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.957548   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.434087   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.747725   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.890597   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...  0.439343   

                                                                                    Joint  \
Group                           Experiment                                                  
joint_training/given_alignments /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   
                                /u/lukas.rilling/experiments/glow_tts_asr_v2/ou...   True   

                                       

In [ ]:
from returnn.config import Config as ReturnnConfig

df_indexed = df.reset_index()

returnn_configs = []
for i in df_indexed.itertuples():
    experiment_path = i[2]
    returnn_config_path = experiment_path.replace("output", "alias") + "training/output/returnn.config"
    returnn_configs.append(ReturnnConfig())
    try:
        returnn_configs[-1].load_file(returnn_config_path)
    except AssertionError:
        returnn_configs[-1] = False

In [ ]:
lr = []
num_epochs = []
asr_model_type = []

for i in df_indexed.itertuples():
    if returnn_configs[i[0]]:
        if returnn_configs[i[0]].has("learning_rates"):
            learning_rates = np.array(eval(returnn_configs[i[0]].value("learning_rates", "default")))
            lr_argmax = learning_rates.argmax()
            lr.append(
                f"[0: {learning_rates[0]}, {lr_argmax}: {learning_rates[lr_argmax]}, {lr_argmax + 1}: {learning_rates[lr_argmax + 1]}, {returnn_configs[i[0]].value('num_epochs', '-')}: {learning_rates[-1]}]"
            )
        elif returnn_configs[i[0]].has("learning_rate"):
            lr.append(returnn_configs[i[0]].value("learning_rate", "-"))
        else:
            lr.append("-")

        num_epochs.append(int(returnn_configs[i[0]].value("num_epochs", "-")))
    else:
        lr.append("-")
        num_epochs.append("-")

    if "conformer" in i[2]:
        asr_model_type.append("conformer")
    elif "blstm" in i[2]:
        asr_model_type.append("blstm")
    else:
        asr_model_type.append("unknown")

df_indexed["Num Epochs"] = num_epochs
df_indexed["LR"] = lr
df_indexed["ASR Model Type"] = asr_model_type

In [ ]:
df_indexed["CE loss scale"] = df_indexed.apply(lambda x: float(x["Experiment"].split("/ce_ls_")[1].split("/")[0] if "/ce_ls_" in x["Experiment"] else np.nan), axis=1)

In [ ]:
df_rounded = df_indexed.replace("-", np.nan).round(decimals=3).fillna("-")
df_rounded.head(5)

,Group,Experiment,Accuracy [%],CE,dev CE,MLE,DP,Joint,Still running,overfitting,Training data available,Num Epochs,LR,ASR Model Type,CE loss scale
0,joint_training/given_alignments,/u/lukas.rilling/experiments/glow_tts_asr_v2/o...,21.30,2.921,2.942,-0.797,0.958,True,False,1.007,True,200.0,"[0: 1e-05, 99: 0.0005, 100: 0.0005, 200: 1e-05]",unknown,0.1
1,joint_training/given_alignments,/u/lukas.rilling/experiments/glow_tts_asr_v2/o...,26.49,2.868,2.859,-0.803,0.434,True,False,0.997,True,250.0,"[0: 7e-06, 109: 0.0007, 110: 0.0007, 250: 1e-08]",unknown,0.1
2,joint_training/given_alignments,/u/lukas.rilling/experiments/glow_tts_asr_v2/o...,27.30,2.664,2.957,-0.721,0.748,True,False,1.11,True,250.0,"[0: 7e-06, 109: 0.0007, 110: 0.0007, 250: 1e-08]",unknown,1.0
3,joint_training/given_alignments,/u/lukas.rilling/experiments/glow_tts_asr_v2/o...,30.09,2.782,2.796,-0.785,0.891,True,False,1.005,True,200.0,"[0: 1e-05, 99: 0.0005, 100: 0.0005, 200: 1e-05]",unknown,0.1
4,joint_training/given_alignments,/u/lukas.rilling/experiments/glow_tts_asr_v2/o...,28.56,2.814,2.825,-0.804,0.439,True,False,1.004,True,200.0,"[0: 1e-05, 99: 0.0005, 100: 0.0005, 200: 1e-05]",unknown,0.1


In [ ]:
print(df_rounded.to_markdown())

|    | Group                           | Experiment                                                                                                                                                                                                                         |   Accuracy [%] | CE    | dev CE   | MLE    | DP    | Joint   | Still running   | overfitting   | Training data available   | Num Epochs   | LR                                               | ASR Model Type   | CE loss scale   |
|---:|:--------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------:|:------|:---------|:-------|:------|:--------|:----------------|:--------------|:--------------------------|:-------------|:-------------------------------------------------|:-----------------|:----------------|
|  0